# 0) Dataload

In [1]:
# === 1. 설정 및 라이브러리 임포트 ===
import pandas as pd
import numpy as np
import random
import os
import itertools
from ast import literal_eval
from collections import defaultdict, Counter
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models import Word2Vec, Doc2Vec
from gensim.models.doc2vec import TaggedDocument

# -----------------------------------------
# [중요] 재현성을 위한 시드(Seed) 고정
# -----------------------------------------
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
os.environ["PYTHONHASHSEED"] = str(SEED)

# 파일 경로 설정 (사용자 환경에 맞게 수정됨)
# DATA_PATH = "/Users/kangmingyun/Desktop/프로젝트/한식레시피/dataset/final/1125final(ing).csv"
DATA_PATH = "C:/Users/User/OneDrive/바탕 화면/한식레시피/dataset/1125final(ing).csv"

print("✅ 라이브러리 임포트 및 시드 고정 완료")

✅ 라이브러리 임포트 및 시드 고정 완료


In [2]:
# === 2. 데이터 로드 ===
if not os.path.exists(DATA_PATH):
    raise FileNotFoundError(f"❌ 파일이 없습니다: {DATA_PATH}")

print(f"📂 데이터 로딩 중: {DATA_PATH}")
df = pd.read_csv(DATA_PATH)

📂 데이터 로딩 중: C:/Users/User/OneDrive/바탕 화면/한식레시피/dataset/1125final(ing).csv


In [3]:
# 불필요한 인덱스 컬럼 제거
if 'Unnamed: 0' in df.columns:
    df = df.drop(columns='Unnamed: 0')

# [핵심] 문자열로 저장된 리스트를 실제 리스트 객체로 변환
# 예: "['마늘', '파']" -> ['마늘', '파']
print("🔄 재료 토큰 리스트 변환 중...")
df['재료토큰'] = df['재료토큰'].apply(literal_eval)

print(f"✅ 데이터 로드 완료! 총 레시피 수: {len(df):,}개")
df

🔄 재료 토큰 리스트 변환 중...
✅ 데이터 로드 완료! 총 레시피 수: 130,801개


,레시피일련번호,요리명,재료통합,요리종류별명_세분화,재료토큰,요리방법별명
0,128671.0,두부새우전,녹말가루 브로콜리 새우 계란 고추 두부 당근,밑반찬,"[녹말가루, 브로콜리, 새우, 계란, 고추, 두부, 당근]",부침
1,128932.0,알밥,계란 노른자 참기름 김가루 후리가케 날치 참깨 당근 단무지 무 밥 신김치,밥,"[계란, 노른자, 참기름, 김가루, 후리가케, 날치, 참깨, 당근, 단무지, 무, ...",굽기
2,131871.0,현미호두죽,물 소금 호두 현미 찹쌀,죽,"[물, 소금, 호두, 현미, 찹쌀]",끓이기
3,139247.0,북어갈비,간장 마늘 파 후춧가루 참기름 설탕 찹쌀가루 물 북어포 깨소금,메인반찬,"[간장, 마늘, 파, 후춧가루, 참기름, 설탕, 찹쌀가루, 물, 북어포, 깨소금]",굽기
4,149207.0,토마토스파게티,올리브유 양파 마늘 토마토 파스타면 피망,면,"[올리브유, 양파, 마늘, 토마토, 파스타면, 피망]",볶음
...,...,...,...,...,...,...
130796,7014698.0,떡볶이,식용유 떡볶이떡 물 어묵 굴 간장 후추 고추장 고추 소금 설탕 대파 마늘,메인반찬,"[식용유, 떡볶이떡, 물, 어묵, 굴, 간장, 후추, 고추장, 고추, 소금, 설탕,...",끓이기
130797,7014700.0,짜장떡볶이,양파 물 마늘 당근 떡볶이 떡 짜장가루 오뎅 올리고당 간장,떡,"[양파, 물, 마늘, 당근, 떡볶이, 떡, 짜장가루, 오뎅, 올리고당, 간장]",볶음
130798,7014702.0,누룽지야채계란죽,파프리카 연두 당근 물 브로콜리 소금 참기름 깨 계란 누룽지 양파,죽,"[파프리카, 연두, 당근, 물, 브로콜리, 소금, 참기름, 깨, 계란, 누룽지, 양파]",끓이기
130799,7014705.0,오징어젓무침,무말랭이 마늘 양파 참깨 참기름 쪽파 고추 오이 오징어젓,젓갈,"[무말랭이, 마늘, 양파, 참깨, 참기름, 쪽파, 고추, 오이, 오징어젓]",무침


In [4]:
df['요리종류별명_세분화'].unique() 

array(['밑반찬', '밥', '죽', '메인반찬', '면', '떡', '만두', '찌개', '소스', '국/탕', '양념',
       '퓨전', '잼', '김치', '젓갈', '장류'], dtype=object)

In [5]:
# 카테고리 통합 매핑 함수
def simplify_category(cat):
    if cat in ['국/탕', '찌개']:
        return '국물요리'
    elif cat in ['밑반찬', '메인반찬', '퓨전']:
        return '일품요리' # 반찬류 통합
    elif cat in ['밥', '죽', '면', '떡', '만두']:
        return '곡물분식'
    elif cat in ['김치', '젓갈', '장류']:
        return '저장음식'
    elif cat in ['양념', '소스', '잼']:
        return '소스류'
    else:
        return '기타'

# 적용
print("🔄 카테고리 단순화 중...")
df['요리종류별명_세분화'] = df['요리종류별명_세분화'].apply(simplify_category)

print("✅ 카테고리 재분류 완료:")
print(df['요리종류별명_세분화'].unique())

🔄 카테고리 단순화 중...
✅ 카테고리 재분류 완료:
['일품요리' '곡물분식' '국물요리' '소스류' '저장음식']


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130801 entries, 0 to 130800
Data columns (total 6 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   레시피일련번호     130801 non-null  float64
 1   요리명         130801 non-null  object 
 2   재료통합        130801 non-null  object 
 3   요리종류별명_세분화  130801 non-null  object 
 4   재료토큰        130801 non-null  object 
 5   요리방법별명      130801 non-null  object 
dtypes: float64(1), object(5)
memory usage: 6.0+ MB


In [7]:
df['요리종류별명_세분화'].unique()

array(['일품요리', '곡물분식', '국물요리', '소스류', '저장음식'], dtype=object)

# 1) Word2vec & Doc2vec 학습

In [9]:
# === 3. 모델 학습 (Word2Vec + Doc2Vec) ===

# 3-1. Word2Vec 학습
print("🚀 Word2Vec 학습 시작... (Window=30, Min=3, Sample=1e-4)")
w2v_model = Word2Vec(
    sentences=df['재료토큰'],
    vector_size=100,
    window=30,        # 레시피 내 모든 재료 관계 학습
    min_count=3,      # 3번 이상 등장하면 학습 (다양성 확보)
    sg=1,             # Skip-gram
    # sample=1e-4,      # 빈출 단어 패널티 (핵심!)
    # ns_exponent=0.75, 
    epochs=30,
    workers=4,
    seed=SEED
)
print(f"✅ Word2Vec 완료 (Vocab: {len(w2v_model.wv.index_to_key):,}개)")

# 3-2. Doc2Vec 학습
print("\n🚀 Doc2Vec 학습 시작... (Tags: Method + Category)")
documents = []
for rid, tokens, method, cat in zip(df['레시피일련번호'], df['재료토큰'], df['요리방법별명'], df['요리종류별명_세분화']):
    # 조리법과 카테고리를 태그로 넣어 문맥에 반영
    words = list(tokens) + [f"METHOD_{method}"] * 2 + [f"CATEGORY_{cat}"] * 2
    tag = f"recipe_{rid}"
    documents.append(TaggedDocument(words=words, tags=[tag]))

d2v_model = Doc2Vec(
    documents,
    vector_size=100,
    window=30,
    min_count=3,
    # sample=1e-4,
    workers=4,
    dm=1, # DM 방식 권장
    epochs=50,
    seed=SEED
)
print(f"✅ Doc2Vec 완료 (Docs: {len(d2v_model.dv):,}개)")

🚀 Word2Vec 학습 시작... (Window=30, Min=3, Sample=1e-4)
✅ Word2Vec 완료 (Vocab: 5,520개)

🚀 Doc2Vec 학습 시작... (Tags: Method + Category)
✅ Doc2Vec 완료 (Docs: 130,735개)


In [19]:
from collections import Counter
import itertools

# 1. 전체 토큰(단어)을 하나로 모아서 개수 세기
all_tokens = list(itertools.chain.from_iterable(df['재료토큰']))
token_counts = Counter(all_tokens)

print(f"🍎 전체 고유 단어(토큰) 개수: {len(token_counts):,}개")
    
# 2. 빈도수 3회 이상인 단어 개수 (Word2Vec이 학습한 대상)
valid_tokens = {word: count for word, count in token_counts.items() if count >= 3}
print(f"✅ 3회 이상 등장한 단어 개수: {len(valid_tokens):,}개")

# 3. Word2Vec 모델의 벡터 개수
print(f"📊 실제 모델에 학습된 벡터 개수: {len(w2v_model.wv.index_to_key):,}개")

# 4. Dc22Vec 모델의 벡터 개수
print(f"📊 실제 모델에 학습된 벡터 개수: {len(d2v_model.dv.index_to_key):,}개")

🍎 전체 고유 단어(토큰) 개수: 18,840개
✅ 3회 이상 등장한 단어 개수: 5,520개
📊 실제 모델에 학습된 벡터 개수: 5,520개
📊 실제 모델에 학습된 벡터 개수: 130,735개


In [20]:
# === 학습 데이터 검증: 버린 단어 vs 챙긴 단어 ===

# 1. 전체 단어 카운트 다시 계산 (위에서 이미 하셨다면 생략 가능)
from collections import Counter
import itertools
all_tokens = list(itertools.chain.from_iterable(df['재료토큰']))
token_counts = Counter(all_tokens)

# 2. 분류 (생존 vs 탈락)
kept_tokens = [word for word, cnt in token_counts.items() if cnt >= 3]
dropped_tokens = [word for word, cnt in token_counts.items() if cnt < 3]

print(f"✅ 살아남은 재료 (예시 30개):")
print(kept_tokens[:30])
print("-" * 50)

print(f"🗑️ 버려진 재료 (예시 30개) - 오타나 너무 특이한 재료인지 보세요:")
print(dropped_tokens[:30])
print("-" * 50)

print(f"📊 요약: 전체 {len(token_counts):,}개 중 {len(kept_tokens):,}개 학습 (삭제율: {len(dropped_tokens)/len(token_counts)*100:.1f}%)")

✅ 살아남은 재료 (예시 30개):
['녹말가루', '브로콜리', '새우', '계란', '고추', '두부', '당근', '노른자', '참기름', '김가루', '후리가케', '날치', '참깨', '단무지', '무', '밥', '신김치', '물', '소금', '호두', '현미', '찹쌀', '간장', '마늘', '파', '후춧가루', '설탕', '찹쌀가루', '북어포', '깨소금']
--------------------------------------------------
🗑️ 버려진 재료 (예시 30개) - 오타나 너무 특이한 재료인지 보세요:
['그외의', '양겨자가루', '헤비위핑크림', '앤젤헤어', '우메보시', '호두아몬드', '중조', '맛있는찌개가필요할때', '박력쌀가루', '청상추', '단호박퓨레', '적양배추', '카스테라가루', '대나무가루', '설탕물에', '졸인', '애느타리버섯', '칠리참치통조림', '짜장면발', '껍질까서', '포도씨유기름', '동글어묵', '초밥유부', '쵸콜릿', '잘익은묵은김치', '매콤한맛굴소스', '맛있는밥&엔', '곤약분말', '깨소슴', '돈까스용등심']
--------------------------------------------------
📊 요약: 전체 18,840개 중 5,520개 학습 (삭제율: 70.7%)


# 2) 인덱스 구축

In [21]:
# === 2. 인덱스 및 통계 구축 ===
print("📊 인덱싱 중...")
name_to_ids = defaultdict(list)
method_map = {}
recipes_by_ingredient = defaultdict(list)
cat_map = {}
ing_cat_counts = defaultdict(Counter)
ing_method_counts = defaultdict(Counter)

for _, row in df.iterrows():
    rid = row['레시피일련번호']
    name = row['요리명']
    method = row['요리방법별명']
    cat = row['요리종류별명_세분화']
    ings = row['재료토큰']
    
    name_to_ids[name].append(rid)
    method_map[rid] = method
    cat_map[rid] = cat
    
    for ing in ings:
        recipes_by_ingredient[ing].append(rid)
        ing_cat_counts[ing][cat] += 1
        ing_method_counts[ing][method] += 1

print("✅ 인덱싱 완료")

📊 인덱싱 중...
✅ 인덱싱 완료


# 3) 대체 로직

## 3-1) 요리종류별명_세분화 & 코사인 유사도 계산 

In [23]:
# -------------------------------------------------------
# 0. Helper Functions (점수 및 유사도 계산)
# -------------------------------------------------------

# [NEW] 범용 통계 점수 계산 (카테고리, 조리법 공용)
# 기존 ing_category_score를 대체합니다.
def get_stat_score(ingredient, target_key, count_dict, min_count=5):
    """
    ingredient: 재료명
    target_key: 현재 요리의 조리법(볶음) 또는 카테고리(반찬)
    count_dict: ing_method_counts 또는 ing_cat_counts
    """
    cnts = count_dict.get(ingredient)
    if not cnts: return 0.0
    total = sum(cnts.values())
    if total < min_count: return 0.0
    return cnts[target_key] / total

# [KEEP] 코사인 유사도 계산 (음수 제거)
# Doc2Vec 벡터 비교 시 사용됩니다.
def cos_sim(vec_a, vec_b):
    norm = (np.linalg.norm(vec_a) * np.linalg.norm(vec_b) + 1e-9)
    return max(0.0, float(np.dot(vec_a, vec_b) / norm))

## 3-2) 단일 대체

In [25]:
# -------------------------------------------------------
# 1. 단일 대체 로직 (Weighted Linear Sum + Optimization)
# -------------------------------------------------------
def substitute_single(recipe_id, target_ing, w_w2v=1.0, w_d2v=1.0, w_method=1.0, w_cat=1.0, topn=10):
    """
    4가지 요소에 대해 가중치를 반영하여 점수를 계산합니다. (정규화 포함)
    """
    # (1) 데이터 준비
    row = df[df['레시피일련번호'] == recipe_id]
    if row.empty: return pd.DataFrame()
    row = row.iloc[0]
    
    current_method = row['요리방법별명']
    current_cat = row['요리종류별명_세분화']
    context_ings = row['재료토큰']
    tag = f"recipe_{recipe_id}"
    
    if target_ing not in w2v_model.wv:
        # print(f"❌ '{target_ing}'은(는) 모델에 없습니다.") # UI 깔끔함을 위해 생략 가능
        return pd.DataFrame()

    # 가중치 총합 계산 (정규화용)
    total_weight = w_w2v + w_d2v + w_method + w_cat
    if total_weight == 0: total_weight = 1.0 # 0 나누기 방지

    # Doc2Vec 벡터 (가중치가 있을 때만 가져옴 -> 속도 최적화)
    vec_recipe = None
    if w_d2v > 0 and tag in d2v_model.dv:
        vec_recipe = d2v_model.dv[tag]
    
    # (2) 후보군 추출
    candidates = w2v_model.wv.most_similar(target_ing, topn=50)
    
    results = []
    for cand, score_w2v in candidates:
        if cand in context_ings: continue
        
        # 1. Word2Vec 점수
        s_w2v = max(0.0, score_w2v)
        if s_w2v < 0.4: continue # [필터] 유사도 0.4 미만 탈락 (엄격하게)
        
        # 2. Doc2Vec 점수 (가중치 > 0 일 때만 계산)
        s_d2v = 0.0
        if w_d2v > 0 and vec_recipe is not None:
            rid_list = recipes_by_ingredient.get(cand, [])
            # 현재 조리법과 같은 레시피들만 추려서 비교
            same_method_ids = [r for r in rid_list if method_map.get(r) == current_method]
            
            # 샘플링 (속도 최적화: 30개)
            if len(same_method_ids) > 30:
                np.random.seed(42)
                same_method_ids = np.random.choice(same_method_ids, 30, replace=False)
            
            if len(same_method_ids) > 0:
                sims = []
                for r in same_method_ids:
                    rt = f"recipe_{r}"
                    if rt in d2v_model.dv:
                        sims.append(cos_sim(vec_recipe, d2v_model.dv[rt]))
                if sims: s_d2v = np.mean(sims)

        # 3. 통계 점수 (가중치 > 0 일 때만 계산)
        s_method = get_stat_score(cand, current_method, ing_method_counts) if w_method > 0 else 0.0
        s_cat = get_stat_score(cand, current_cat, ing_cat_counts) if w_cat > 0 else 0.0
        
        # (3) 최종 점수 계산 (가중 합산 후 정규화)
        weighted_sum = (w_w2v * s_w2v) + (w_d2v * s_d2v) + (w_method * s_method) + (w_cat * s_cat)
        final_score = weighted_sum / total_weight
        
        results.append({
            "대체재료": cand,
            "최종점수": final_score,
            "W2V": s_w2v,
            "D2V": s_d2v,
            "Method": s_method,
            "Category": s_cat
        })
        
    # (4) 결과 반환
    df_res = pd.DataFrame(results)
    if df_res.empty: return df_res
    
    return df_res.sort_values("최종점수", ascending=False).head(topn).reset_index(drop=True)

print("✅ 단일 대체 로직(최적화+정규화) 준비 완료")

✅ 단일 대체 로직(최적화+정규화) 준비 완료


## 3-3) 다중 대체

In [26]:
# -------------------------------------------------------
# 2. 다중 대체 로직 (Beam Search + Weighted Sum)
# -------------------------------------------------------
def substitute_multi(recipe_id, targets, w_w2v=1.0, w_d2v=1.0, w_method=1.0, w_cat=1.0, w_cluster=0.0, beam_width=3):
    """
    순차적 조건부 방식을 적용하여 최적의 재료 조합을 찾습니다. (Beam Search)
    """
    # (1) 초기 데이터 준비
    row = df[df['레시피일련번호'] == recipe_id]
    if row.empty: return []
    row = row.iloc[0]
    
    current_method = row['요리방법별명']
    current_cat = row['요리종류별명_세분화']
    
    # 초기 문맥 (원래 레시피 재료들)
    initial_context = row['재료토큰']
    
    tag = f"recipe_{recipe_id}"
    # Doc2Vec 벡터 (가중치 있을 때만 로드)
    vec_recipe = None
    if w_d2v > 0 and tag in d2v_model.dv:
        vec_recipe = d2v_model.dv[tag]
    
    # 가중치 총합 (정규화용)
    total_weight = w_w2v + w_d2v + w_method + w_cat + w_cluster
    if total_weight == 0: total_weight = 1.0

    # Beam 초기화: [(누적점수, [대체재료리스트], [현재문맥리스트])]
    beam = [(0.0, [], initial_context)]
    
    # (2) 타겟 재료 순차 처리
    for target_ing in targets:
        next_beam = []
        
        if target_ing not in w2v_model.wv:
            # 모델에 없으면 그냥 스킵 (대체 불가)
            for score, subs, ctx in beam:
                next_beam.append((score, subs + [target_ing], ctx))
            beam = next_beam
            continue
            
        # 타겟의 클러스터 ID (필터링용)
        target_cluster = ingredient_clusters.get(target_ing, -1)

        for path_score, path_subs, path_ctx in beam:
            # 현재 문맥에서 타겟 제거
            current_ctx_ing = [x for x in path_ctx if x != target_ing]
            
            # 후보군 추출 (Top 20)
            candidates = w2v_model.wv.most_similar(target_ing, topn=20)
            
            for cand, sim_orig in candidates:
                if cand in current_ctx_ing or cand in path_subs: continue
                
                sim_orig = max(0.0, sim_orig)
                if sim_orig < 0.3: continue
                
                # [점수 계산]
                
                # A. W2V (유사도 + 문맥 조화도)
                # 변경된 문맥과의 평균 유사도 계산 (Harmony)
                harmony_scores = [w2v_model.wv.similarity(cand, c) for c in current_ctx_ing if c in w2v_model.wv]
                sim_harmony = np.mean(harmony_scores) if harmony_scores else 0.0
                sim_harmony = max(0.0, sim_harmony)
                
                # W2V 점수는 (원래 재료 유사도 50% + 새 문맥 조화도 50%)
                s_w2v = 0.5 * sim_orig + 0.5 * sim_harmony
                
                # B. D2V (최적화 적용)
                s_d2v = 0.0
                if vec_recipe is not None:
                    rid_list = recipes_by_ingredient.get(cand, [])
                    same_method_ids = [r for r in rid_list if method_map.get(r) == current_method]
                    
                    if len(same_method_ids) > 20:
                        np.random.seed(42)
                        same_method_ids = np.random.choice(same_method_ids, 20, replace=False)
                    
                    if len(same_method_ids) > 0:
                        sims = []
                        for r in same_method_ids:
                            rt = f"recipe_{r}"
                            if rt in d2v_model.dv:
                                sims.append(cos_sim(vec_recipe, d2v_model.dv[rt]))
                        if sims: s_d2v = np.mean(sims)
                
                # C. Stats
                s_method = get_stat_score(cand, current_method, ing_method_counts) if w_method > 0 else 0.0
                s_cat = get_stat_score(cand, current_cat, ing_cat_counts) if w_cat > 0 else 0.0
                
                # D. Cluster
                cand_cluster = ingredient_clusters.get(cand, -2)
                s_cluster = 1.0 if target_cluster == cand_cluster else 0.0
                
                # E. 최종 점수 (가중 합산 & 정규화)
                weighted_sum = (w_w2v * s_w2v) + (w_d2v * s_d2v) + \
                               (w_method * s_method) + (w_cat * s_cat) + \
                               (w_cluster * s_cluster)
                
                step_score = weighted_sum / total_weight
                
                # F. 경로 업데이트
                new_total_score = path_score + step_score
                new_subs = path_subs + [cand]
                new_ctx = current_ctx_ing + [cand] # 문맥 업데이트!
                
                next_beam.append((new_total_score, new_subs, new_ctx))
        
        # Beam Pruning (Top N 유지)
        next_beam.sort(key=lambda x: x[0], reverse=True)
        beam = next_beam[:beam_width]
    
    # (3) 결과 포맷팅
    results = []
    for score, subs, _ in beam:
        avg_score = score / len(targets) if targets else 0.0
        results.append((subs, avg_score))
        
    return results

print("✅ 다중 대체 로직(Beam Search+최적화) 준비 완료")

✅ 다중 대체 로직(Beam Search+최적화) 준비 완료


# 4) UI

In [ ]:
# === 12. [Final UI] 수식 시각화 & 로직 연동 대시보드 ===
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML, Markdown
import pandas as pd

# -------------------------------------------------------
# UI 구성
# -------------------------------------------------------
input_dish = widgets.Text(description="요리명:", placeholder="예: 김치찌개", layout=widgets.Layout(width='40%'))
btn_search = widgets.Button(description="검색", button_style='info', icon='search', layout=widgets.Layout(width='15%'))
dropdown_recipe = widgets.Dropdown(description="레시피:", options=[], disabled=True, layout=widgets.Layout(width='99%'))
input_target = widgets.Text(description="타겟 재료:", placeholder="예: 돼지고기, 양파 (쉼표 구분)", disabled=True, layout=widgets.Layout(width='50%'))

style = {'description_width': 'initial'}
s_w2v = widgets.FloatSlider(value=1.0, min=0.0, max=5.0, step=0.5, description='맛·성질(W2V)', style=style)
s_d2v = widgets.FloatSlider(value=1.0, min=0.0, max=5.0, step=0.5, description='문맥(D2V)', style=style)
s_meth = widgets.FloatSlider(value=1.0, min=0.0, max=5.0, step=0.5, description='조리법', style=style)
s_cat = widgets.FloatSlider(value=1.0, min=0.0, max=5.0, step=0.5, description='카테고리', style=style)

btn_run = widgets.Button(description="분석 실행", button_style='primary', disabled=True, icon='rocket', layout=widgets.Layout(width='100%'))
output_main = widgets.Output(layout=widgets.Layout(width='100%'))

# -------------------------------------------------------
# UI 업데이트 로직
# -------------------------------------------------------
def update_dashboard(recipe_id, target_input_str, w_w2v, w_d2v, w_method, w_cat):
    
    raw_targets = target_input_str.replace(',', ' ').split()
    targets = [t.strip() for t in raw_targets if t.strip()]
    if not targets: return
    
    # [NEW] 수식 시각화: 현재 가중치 상태를 수식으로 보여줌
    total_w = w_w2v + w_d2v + w_method + w_cat
    if total_w == 0: total_w = 1.0
    
    formula_md = f"""
    <div style="background-color: #f0f8ff; padding: 10px; border-radius: 5px; margin-bottom: 15px;">
        <h4 style="margin:0;">🧮 실시간 점수 산출 수식</h4>
        <p style="font-family: monospace; font-size: 1.1em; margin: 5px 0;">
            Score = ( {w_w2v}×W2V + {w_d2v}×D2V + {w_method}×Method + {w_cat}×Category ) / {total_w:.1f}
        </p>
    </div>
    """
    display(HTML(formula_md))
    
    # 1. 단일 대체 결과 출력
    cached_results = {}
    valid_targets = []
    html_out = "<div style='display: flex; flex-wrap: wrap; gap: 20px; margin-bottom: 10px; width: 100%;'>"
    
    for t in targets:
        # [외부 로직 호출] substitute_single
        res = substitute_single(recipe_id, t, w_w2v, w_d2v, w_method, w_cat, topn=5)
        
        if not res.empty:
            cached_results[t] = res
            valid_targets.append(t)
            
            # 테이블 스타일링
            styler = res.style.format("{:.3f}", subset=["최종점수", "W2V", "D2V", "Method", "Category"]) \
                        .background_gradient(cmap='Greens', subset=["최종점수"]) \
                        .set_caption(f"<b>🔹 {t} 대체 추천</b>") \
                        .set_table_attributes('style="border:1px solid #ccc; border-collapse:collapse;"')
            html_out += f"<div style='flex: 1; min-width: 300px;'>{styler.to_html()}</div>"
        else:
            html_out += f"<div><b>{t}:</b> ❌ 결과 없음</div>"
            
    html_out += "</div>"
    display(HTML(html_out))
    
    # 2. 다중 대체 결과 출력
    if len(targets) > 1 and valid_targets:
        display(Markdown("### 🧩 최적의 재료 조합 (다중 대체)"))
        # [외부 로직 호출] substitute_multi
        multi_res = substitute_multi(recipe_id, targets, w_w2v, w_d2v, w_method, w_cat, beam_width=3)
        
        if multi_res:
            m_df = pd.DataFrame([
                (f"{', '.join(subs)}", f"{score:.3f}") for subs, score in multi_res
            ], columns=['추천 조합', '평균 점수'])
            display(m_df.style.background_gradient(cmap='Blues', subset=['평균 점수']))

# -------------------------------------------------------
# 이벤트 핸들러
# -------------------------------------------------------
def on_search_click(b):
    output_main.clear_output(); dropdown_recipe.options = []
    dish = input_dish.value.strip()
    if not dish: return
    
    cands = df[df['요리명'] == dish]
    if cands.empty: cands = df[df['요리명'].str.contains(dish, na=False)]
    cands = cands.head(10).reset_index(drop=True)
    
    if cands.empty:
        with output_main: print("❌ 검색 결과 없음"); return
        
    opts = []
    for _, r in cands.iterrows():
        ing_sum = ', '.join(r['재료토큰'])
        label = f"[{r['요리종류별명_세분화']}/{r['요리방법별명']}] {r['요리명']} (ID:{r['레시피일련번호']}) - {ing_sum[:50]}..."
        opts.append((label, r['레시피일련번호']))
    
    dropdown_recipe.options = opts
    dropdown_recipe.disabled = False; input_target.disabled = False; btn_run.disabled = False
    with output_main: print(f"✅ '{dish}' 검색 완료")

def update_result(b=None):
    if dropdown_recipe.disabled: return
    recipe_id = dropdown_recipe.value
    target_str = input_target.value.strip()
    if not target_str: return
    
    with output_main:
        clear_output(wait=True)
        update_dashboard(
            recipe_id, target_str,
            s_w2v.value, s_d2v.value, s_meth.value, s_cat.value
        )

# 연결
btn_search.on_click(on_search_click)
btn_run.on_click(update_result)
# 슬라이더 변경 시 실시간 업데이트
for s in [s_w2v, s_d2v, s_meth, s_cat]: 
    s.observe(lambda c: update_result(), names='value')

ui = widgets.VBox([
    lbl_title,
    widgets.HBox([input_dish, btn_search], layout=widgets.Layout(width='100%')),
    dropdown_recipe,
    input_target,
    widgets.HTML("<b>⚖️ 가중치 설정 (0.0 ~ 5.0)</b>"),
    widgets.HBox([s_w2v, s_d2v, s_meth, s_cat], layout=widgets.Layout(width='100%')),
    btn_run,
    widgets.HTML("<hr>"),
    output_main
], layout=widgets.Layout(width='99%', padding='10px'))

display(ui)

In [ ]:
# === 12. [Final UI] 4대 가중치 조절 & 다중 시각화 대시보드 ===
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML, Markdown
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.decomposition import PCA
import pandas as pd
import numpy as np

# 전역 변수
dashboard_widget = None
cached_results = {} # {재료명: 결과DF} 저장

# -------------------------------------------------------
# 1. [Visualization Helper] 시각화용 데이터 계산
#    (UI 전용 함수이므로 여기에 둡니다)
# -------------------------------------------------------
def calc_radar_data(ingredient, target_ing, recipe_id, w_method, w_cat):
    """레이더 차트용 4축 데이터 계산"""
    row = df[df['레시피일련번호'] == recipe_id].iloc[0]
    
    # 1. 재료유사도 (W2V)
    try: sim_w2v = max(0.0, float(w2v_model.wv.similarity(target_ing, ingredient)))
    except: sim_w2v = 0.0
    
    # 2. 문맥유사도 (D2V Proxy: 문맥 재료들과의 평균 유사도)
    context = [i for i in row['재료토큰'] if i != target_ing and i in w2v_model.wv]
    sim_d2v = 0.0
    if context:
        sim_d2v = max(0.0, float(np.mean([w2v_model.wv.similarity(ingredient, c) for c in context])))
    
    # 3. 조리법 유사도
    score_method = 0.0
    if w_method > 0: # 가중치가 있을 때만 계산
        score_method = get_stat_score(ingredient, row['요리방법별명'], ing_method_counts)
        
    # 4. 카테고리 유사도
    score_cat = 0.0
    if w_cat > 0:
        score_cat = get_stat_score(ingredient, row['요리종류별명_세분화'], ing_cat_counts)
        
    return [sim_w2v, sim_d2v, score_method, score_cat]

# -------------------------------------------------------
# 2. [UI Logic] 대시보드 업데이트
# -------------------------------------------------------
def update_dashboard(recipe_id, target_input_str, w_w2v, w_d2v, w_method, w_cat):
    global dashboard_widget, cached_results
    
    # 1. 입력 파싱
    raw_targets = target_input_str.replace(',', ' ').split()
    targets = [t.strip() for t in raw_targets if t.strip()]
    if not targets: return
    
    # 2. [Logic Call] 단일 대체 로직 호출 (각 타겟별)
    cached_results = {}
    valid_targets = []
    
    # 가로형 테이블 출력을 위한 HTML
    html_out = "<div style='display: flex; flex-wrap: wrap; gap: 20px; margin-bottom: 10px; width: 100%;'>"
    
    for t in targets:
        # 앞서 정의한 substitute_single 함수 호출
        res = substitute_single(recipe_id, t, w_w2v, w_d2v, w_method, w_cat, topn=5)
        
        if not res.empty:
            cached_results[t] = res
            valid_targets.append(t)
            
            # 테이블 스타일링
            styler = res.style.format("{:.3f}", subset=["최종점수", "W2V", "D2V", "Method", "Category"]) \
                        .background_gradient(cmap='Greens', subset=["최종점수"]) \
                        .set_caption(f"<b>🔹 {t} 대체 추천</b>") \
                        .set_table_attributes('style="border:1px solid #ccc; border-collapse:collapse;"')
            html_out += f"<div style='flex: 1; min-width: 300px;'>{styler.to_html()}</div>"
        else:
            html_out += f"<div><b>{t}:</b> ❌ 결과 없음</div>"
            
    html_out += "</div>"
    display(HTML(html_out))
    
    # 3. [Logic Call] 다중 대체 로직 호출 (2개 이상일 때)
    if len(targets) > 1 and valid_targets:
        display(Markdown("### 🧩 최적의 재료 조합 (다중 대체)"))
        multi_res = substitute_multi(recipe_id, targets, w_w2v, w_d2v, w_method, w_cat, result_topn=3)
        if multi_res:
            m_df = pd.DataFrame([
                (f"{', '.join(subs)}", f"{score:.3f}") for subs, score in multi_res
            ], columns=['추천 조합', '평균 점수'])
            display(m_df.style.background_gradient(cmap='Blues', subset=['평균 점수']))
    
    if not valid_targets: return

    # --- [Visualization] 데이터 준비 ---
    # 시각화 대상: 드롭다운 값 (없으면 첫 번째 타겟)
    vis_target = dropdown_vis.value if dropdown_vis.value in valid_targets else valid_targets[0]
    
    # 드롭다운 옵션 갱신
    dropdown_vis.options = valid_targets
    dropdown_vis.value = vis_target
    dropdown_vis.disabled = False

    row = df[df['레시피일련번호'] == recipe_id].iloc[0]
    # 문맥 (타겟들 제외)
    context_ings = [ing for ing in row['재료토큰'] if ing not in targets and ing in w2v_model.wv]
    
    # 3D 시각화용 벡터 수집 (모든 타겟 + 각각의 1등 + 문맥)
    all_words = []
    target_best_pairs = [] # (타겟, 1등)
    
    for t in valid_targets:
        best_sub = cached_results[t].iloc[0]['대체재료']
        target_best_pairs.append((t, best_sub))
        all_words.extend([t, best_sub])
        
    all_words.extend(context_ings)
    
    vectors = [w2v_model.wv[w] for w in all_words]
    if len(vectors) < 3: return # 시각화 불가
    
    pca = PCA(n_components=3)
    vecs_3d = pca.fit_transform(vectors)
    
    # 좌표 매핑
    ctx_start_idx = len(target_best_pairs) * 2
    ctx_3d = vecs_3d[ctx_start_idx:]
    
    # 레이더/바 차트 데이터 (선택된 vis_target 기준)
    vis_best = cached_results[vis_target].iloc[0]['대체재료']
    metrics_target = calc_radar_data(vis_target, vis_target, recipe_id, w_method, w_cat)
    metrics_best   = calc_radar_data(vis_best, vis_target, recipe_id, w_method, w_cat)
    radar_cats = ['재료유사', '문맥유사', '조리법', '카테고리']
    
    harmony_scores = [float(w2v_model.wv.similarity(vis_best, c)) for c in context_ings]
    
    # --- [Visualization] 그래프 업데이트 ---
    if dashboard_widget is None:
        # 위젯 초기 생성
        fig = make_subplots(
            rows=2, cols=2,
            specs=[[{"type": "scene", "rowspan": 2}, {"type": "polar"}], [None, {"type": "xy"}]],
            column_widths=[0.60, 0.40], horizontal_spacing=0.02, vertical_spacing=0.08,
            subplot_titles=("🌌 재료 3D 벡터 공간 (All Targets)", "📊 상세 분석 (선택 재료)", "🎹 문맥 조화도 (선택 재료)")
        )
        # Traces
        fig.add_trace(go.Scatter3d(mode='markers', name='Context'), row=1, col=1) #0
        fig.add_trace(go.Scatter3d(mode='lines', name='Links'), row=1, col=1)     #1
        fig.add_trace(go.Scatter3d(mode='markers+text', name='Targets'), row=1, col=1) #2
        fig.add_trace(go.Scatter3d(mode='markers+text', name='Bests'), row=1, col=1)   #3
        fig.add_trace(go.Scatterpolar(fill='toself', name='Target Spec'), row=1, col=2) #4
        fig.add_trace(go.Scatterpolar(fill='toself', name='Best Spec'), row=1, col=2)   #5
        fig.add_trace(go.Bar(orientation='h', name='Harmony'), row=2, col=2)            #6
        
        # Layout: Autoscale & Full Fill
        fig.update_layout(
            template="plotly_dark", height=550, autosize=True,
            margin=dict(t=40, l=0, r=0, b=0),
            showlegend=True, legend=dict(x=0, y=1, font=dict(size=10)),
            font=dict(family="Malgun Gothic, AppleGothic, sans-serif", size=11)
        )
        fig.update_scenes(xaxis_visible=False, yaxis_visible=False, zaxis_visible=False, aspectmode='data')
        fig.update_polars(radialaxis=dict(visible=True, range=[0, 1], showticklabels=True))
        
        dashboard_widget = go.FigureWidget(fig)
        dashboard_widget.layout.width = '100%' # 가로 꽉 채움

    with dashboard_widget.batch_update():
        dashboard_widget.layout.title.text = f"<b>[{row['요리명']}] {target_input_str} 대체 분석</b>"
        
        # 3D Context
        dashboard_widget.data[0].x=ctx_3d[:,0]; dashboard_widget.data[0].y=ctx_3d[:,1]; dashboard_widget.data[0].z=ctx_3d[:,2]
        dashboard_widget.data[0].text=context_ings; dashboard_widget.data[0].marker.color='rgba(255,255,255,0.1)'; dashboard_widget.data[0].marker.size=3
        
        # 3D Targets & Bests & Lines
        lx, ly, lz = [], [], []
        tx, ty, tz, tt = [], [], [], []
        bx, by, bz, bt = [], [], [], []
        
        for i, (t, b) in enumerate(target_best_pairs):
            t_vec = vecs_3d[i*2]
            b_vec = vecs_3d[i*2+1]
            
            # Line
            lx.extend([t_vec[0], b_vec[0], None]); ly.extend([t_vec[1], b_vec[1], None]); lz.extend([t_vec[2], b_vec[2], None])
            # Target Marker
            tx.append(t_vec[0]); ty.append(t_vec[1]); tz.append(t_vec[2]); tt.append(f"Target: {t}")
            # Best Marker
            bx.append(b_vec[0]); by.append(b_vec[1]); bz.append(b_vec[2]); bt.append(f"★ {b}")
            
        dashboard_widget.data[1].x=lx; dashboard_widget.data[1].y=ly; dashboard_widget.data[1].z=lz
        dashboard_widget.data[1].line.color='#ffd700'; dashboard_widget.data[1].line.width=4
        
        dashboard_widget.data[2].x=tx; dashboard_widget.data[2].y=ty; dashboard_widget.data[2].z=tz
        dashboard_widget.data[2].text=tt; dashboard_widget.data[2].marker.color='#ff3366'; dashboard_widget.data[2].marker.size=10
        
        dashboard_widget.data[3].x=bx; dashboard_widget.data[3].y=by; dashboard_widget.data[3].z=bz
        dashboard_widget.data[3].text=bt; dashboard_widget.data[3].marker.color='#00ff99'; dashboard_widget.data[3].marker.size=15; dashboard_widget.data[3].marker.symbol='diamond'
        
        # Radar
        dashboard_widget.data[4].r=metrics_target; dashboard_widget.data[4].theta=radar_cats; dashboard_widget.data[4].line.color='#ff3366'; dashboard_widget.data[4].name=f"Target: {vis_target}"
        dashboard_widget.data[5].r=metrics_best; dashboard_widget.data[5].theta=radar_cats; dashboard_widget.data[5].line.color='#00ff99'; dashboard_widget.data[5].name=f"Best: {vis_best}"
        
        # Bar
        dashboard_widget.data[6].x=harmony_scores; dashboard_widget.data[6].y=context_ings; dashboard_widget.data[6].marker.color=harmony_scores; dashboard_widget.data[6].marker.colorscale='Viridis'

# -------------------------------------------------------
# 3. UI Layout Construction
# -------------------------------------------------------
# 1) 입력부
lbl_title = widgets.HTML("<h2>🍳 AI 식재료 대체 추천 대시보드</h2>")
input_dish = widgets.Text(description="요리명:", placeholder="예: 김치찌개", layout=widgets.Layout(width='40%'))
btn_search = widgets.Button(description="검색", button_style='info', icon='search', layout=widgets.Layout(width='15%'))
dropdown_recipe = widgets.Dropdown(description="레시피:", options=[], disabled=True, layout=widgets.Layout(width='99%'))
input_target = widgets.Text(description="타겟 재료:", placeholder="예: 돼지고기, 양파 (쉼표 구분)", disabled=True, layout=widgets.Layout(width='50%'))

# 2) 제어부 (가중치 슬라이더)
style = {'description_width': 'initial'}
s_w2v = widgets.FloatSlider(value=1.0, min=0.0, max=5.0, step=0.1, description='맛·성질(W2V)', style=style, layout=widgets.Layout(width='24%'))
s_d2v = widgets.FloatSlider(value=1.0, min=0.0, max=5.0, step=0.1, description='문맥(D2V)', style=style, layout=widgets.Layout(width='24%'))
s_meth = widgets.FloatSlider(value=1.0, min=0.0, max=5.0, step=0.1, description='조리법', style=style, layout=widgets.Layout(width='24%'))
s_cat = widgets.FloatSlider(value=1.0, min=0.0, max=5.0, step=0.1, description='카테고리', style=style, layout=widgets.Layout(width='24%'))

# 3) 실행부
btn_run = widgets.Button(description="분석 실행", button_style='primary', disabled=True, icon='rocket', layout=widgets.Layout(width='100%'))
dropdown_vis = widgets.Dropdown(description="👁️ 상세 시각화:", options=[], disabled=True, layout=widgets.Layout(width='50%'))

# 4) 출력부
output_main = widgets.Output(layout=widgets.Layout(width='100%'))

# -------------------------------------------------------
# 4. 이벤트 핸들러 연결
# -------------------------------------------------------
def on_search_click(b):
    output_main.clear_output(); dropdown_recipe.options = []
    dish = input_dish.value.strip()
    if not dish: return
    
    cands = df[df['요리명'] == dish]
    if cands.empty: cands = df[df['요리명'].str.contains(dish, na=False)]
    cands = cands.head(10).reset_index(drop=True)
    
    if cands.empty:
        with output_main: print("❌ 검색 결과 없음")
        return
        
    opts = []
    for _, r in cands.iterrows():
        ing_sum = ', '.join(r['재료토큰'])[:60] + "..."
        opts.append((f"[{r['요리종류별명_세분화']}/{r['요리방법별명']}] {r['요리명']} (ID:{r['레시피일련번호']}) - {ing_sum}", r['레시피일련번호']))
        
    dropdown_recipe.options = opts
    dropdown_recipe.disabled = False; input_target.disabled = False; btn_run.disabled = False
    with output_main: print(f"✅ '{dish}' 검색 완료. 레시피를 선택하고 재료를 입력하세요.")

def trigger_analysis(b=None):
    if dropdown_recipe.disabled: return
    with output_main:
        clear_output(wait=True)
        update_dashboard(
            dropdown_recipe.value, input_target.value.strip(),
            s_w2v.value, s_d2v.value, s_meth.value, s_cat.value
        )
        if dashboard_widget: display(dashboard_widget)

def on_vis_dropdown_change(change):
    # 드롭다운 변경 시 해당 재료로 차트만 업데이트 (재계산 X)
    if change['new']:
        # 현재 설정된 가중치로 차트 데이터 재계산하여 표시
        # (편의상 trigger_analysis를 호출하여 전체 갱신)
        trigger_analysis()

btn_search.on_click(on_search_click)
btn_run.on_click(trigger_analysis)
# 슬라이더 변경 시 실시간 반영
for s in [s_w2v, s_d2v, s_meth, s_cat]:
    s.observe(lambda c: trigger_analysis(), names='value')
dropdown_vis.observe(on_vis_dropdown_change, names='value')

# -------------------------------------------------------
# 5. 레이아웃 조립
# -------------------------------------------------------
ui = widgets.VBox([
    lbl_title,
    widgets.HBox([input_dish, btn_search], layout=widgets.Layout(width='100%')),
    dropdown_recipe,
    widgets.HBox([input_target, dropdown_vis], layout=widgets.Layout(width='100%')),
    widgets.HTML("<b>⚖️ 가중치 설정 (0.0 ~ 5.0)</b>"),
    widgets.HBox([s_w2v, s_d2v, s_meth, s_cat], layout=widgets.Layout(width='100%')),
    btn_run,
    widgets.HTML("<hr>"),
    output_main
], layout=widgets.Layout(width='99%'))

display(ui)

# 4) 실행 로직

In [82]:
# === 6. 실행 인터페이스 (검색 로직 개선: 완전일치 우선) ===

def run():
    print("\n🍳 [시스템] 요리 재료 대체 프로그램을 시작합니다.")
    while True:
        # 1. 요리명 입력
        dish_name = input("\n🍽  요리명을 입력하세요 (예: 김치찌개) [종료: q]: ").strip()
        if dish_name == 'q': break
        
        if not dish_name:
            print("❌ 요리명을 입력해주세요.")
            continue
        
        # 2. 요리 검색 로직 (우선순위 적용)
        # (1단계) 완전 일치하는 요리 찾기
        candidates = df[df['요리명'] == dish_name]
        
        # (2단계) 완전 일치가 없으면, 포함된 요리 찾기 (Contains)
        if candidates.empty:
            candidates = df[df['요리명'].str.contains(dish_name, na=False)]
            
        if candidates.empty:
            print(f"❌ '{dish_name}'에 해당하는 요리를 찾을 수 없습니다.")
            continue
            
        # 3. 검색 결과 목록 출력 (최대 10개)
        # 인덱스를 초기화하여 0번부터 시작하게 맞춤
        candidates = candidates.head(10).reset_index(drop=True)
        
        print(f"\n🧾 '{dish_name}' 검색 결과 (상위 {len(candidates)}개):")
        print("-" * 70)
        for i, row in candidates.iterrows():
            # 재료가 너무 길면 잘라서 보여줌
            ing_preview = ', '.join(row['재료토큰'][:6]) + "..." if len(row['재료토큰']) > 6 else ', '.join(row['재료토큰'])
            
            print(f"[{i+1}] {row['요리명']} (ID:{row['레시피일련번호']})")
            print(f"    - 조리법: {row['요리방법별명']} | 카테고리: {row['요리종류별명_세분화']}")
            print(f"    - 재료: {ing_preview}")
            print("-" * 70)
            
        # 4. 사용자 선택 (번호 입력)
        while True:
            sel = input(f"👉 사용할 레시피 번호를 선택하세요 (1~{len(candidates)}): ").strip()
            if sel.isdigit() and 1 <= int(sel) <= len(candidates):
                selected_idx = int(sel) - 1 # 0-based index로 변환
                break
            print("❌ 잘못된 입력입니다. 목록에 있는 번호를 입력해주세요.")
            
        # 선택된 레시피 정보 확정
        row = candidates.iloc[selected_idx]
        recipe_id = row['레시피일련번호']
        
        print(f"\n✅ [선택 완료] {row['요리명']} (ID: {recipe_id})")
        print(f"📝 전체 재료: {', '.join(row['재료토큰'])}")
        print("=" * 70)
        
        # 5. 대체 모드 실행
        mode = input("👉 모드 선택 (1: 단일 대체 / 2: 다중 대체): ").strip()
        
        if mode == '1':
            target = input("🎯 바꿀 재료 입력: ").strip()
            res = substitute_single(recipe_id, target)
            if not res.empty:
                print("\n[추천 결과]")
                # 주피터 환경이면 display 사용, 아니면 print
                display(res) if 'display' in globals() else print(res)
            else:
                print("❌ 추천 결과가 없습니다.")
                
        elif mode == '2':
            targets = input("🎯 바꿀 재료들 입력 (공백 구분): ").split()
            res = substitute_multi(recipe_id, targets)
            if res:
                print("\n[추천 조합]")
                for i, (combo, w, d, s) in enumerate(res, 1):
                    print(f"{i}. {', '.join(combo)} (점수: {s:.3f})")
            else:
                print("❌ 추천 결과가 없습니다.")

# 실행
run()


🍳 [시스템] 요리 재료 대체 프로그램을 시작합니다.
❌ '김치지개'에 해당하는 요리를 찾을 수 없습니다.

🧾 '김치' 검색 결과 (상위 10개):
----------------------------------------------------------------------
[1] 김치 (ID:1555483.0)
    - 조리법: 절임 | 카테고리: 김치
    - 재료: 까나리, 생강, 설탕, 천일염소금, 마늘, 고추...
----------------------------------------------------------------------
[2] 김치 (ID:2358033.0)
    - 조리법: 절임 | 카테고리: 김치
    - 재료: 설탕, 새우젓, 액젓, 마늘, 생강, 고추...
----------------------------------------------------------------------
[3] 김치 (ID:4849802.0)
    - 조리법: 절임 | 카테고리: 김치
    - 재료: 대파, 고추, 토마토, 양파, 야콘, 고려엉겅퀴...
----------------------------------------------------------------------
[4] 김치 (ID:5314222.0)
    - 조리법: 절임 | 카테고리: 김치
    - 재료: 고려엉겅퀴, 찹쌀가루, 고추, 생강, 마늘, 참깨...
----------------------------------------------------------------------
[5] 김치 (ID:6835983.0)
    - 조리법: 무침 | 카테고리: 김치
    - 재료: 설탕, 참깨, 참기름, 묵은김치
----------------------------------------------------------------------
[6] 김치 (ID:6850820.0)
    - 조리법: 무침 | 카테고리: 김치
    - 재료: 통마늘, 고추, MS